## Loading packages, constants, and functions

In [1]:
import functions as f
import constants as c
import classes as cl

import pandas as pd
import os, joblib, time
print(f"Current directories in {c.cwd}\n {os.listdir()}")

long = c.cwd + "\\raw_data\\long_alzheimers.tsv"
short = c.cwd + "\\raw_data\\short_alzheimers.tsv"

c:\Users\tobia\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\tobia\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Current directories in c:\Users\tobia\Vs code\02807\proj\02807_project_Group13\
 ['.git', 'classes.py', 'constants.py', 'functions.py', 'joblib_vars', 'Notebook.ipynb', 'raw_data', 'README.md', '__pycache__']


## Loading Datasets

#### Creating id_prot and saving
id_prot is a dictionary meant to point a id <integer> to a protein name <string> with 19 str characters \
id_prot is saved using joblib.dump() in dir "joblib_vars", and can be reloaded as a dict object using joblib.load(path)

In [ ]:
chunksize = 10
info_df = pd.read_csv("https://stringdb-downloads.org/download/protein.info.v12.0/9606.protein.info.v12.0.txt.gz", compression='gzip', sep="\t")
info_df = info_df[["#string_protein_id"]].values
id_prot = dict()
for c, e in enumerate(info_df):
    id_prot[c] = e[0]
print(id_prot)
joblib.dump(id_prot, "./joblib_vars/id_prot.joblib")

#### Modifying raw datasets
modified datasets are stored in "/mod_data/" for easy retrieval

In [ ]:
"""
An example of an interaction dataset loaded
"""
alz_int_df = pd.read_csv("./raw_data/long_alzheimers.tsv", sep="\t")
alz_int_df.columns = ["Prot1_ShortName", "Prot2_ShortName", "protein1", "protein2", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9"]
alz_int_df

In [ ]:
HS_int_df = pd.read_csv("https://stringdb-downloads.org/download/protein.links.detailed.v12.0/9606.protein.links.detailed.v12.0.txt.gz", compression="gzip", sep=" ")
HS_int_df = HS_int_df[HS_int_df["experimental"] > 0] #removing interactions with no experimental relevance 
#joblib.dump(HS_int_df, "./joblib_vars/HS_int_df") #FILE TOO LARGE 
HS_int_df


Creating HS_int_simple, containing protein1 and protein2 names converted to int using id_prot. \
If all interactions are weight=1, only the names (ids) of the proteins are needed when constructing the interaction network

In [ ]:
id_prot = joblib.load("./joblib_vars/id_prot.joblib")

#Swapping key value in dict
id_swap = dict()
for k, v in id_prot.items():
    id_swap[str(v)] = k
print(id_swap)

#exporting modified HS_int
HS_int_simple = HS_int_df.map(lambda x: id_swap[x] if x in id_swap else x).reset_index()
HS_int_simple = HS_int_simple[["protein1", "protein2"]]
joblib.dump(HS_int_simple, "./joblib_vars/HS_int_simple.joblib")


## Nodes & Edges creation from datasets

In [ ]:
"""for a in HS_int_simple.iterrows():
    print(a[1]["protein1"], a[1]["protein2"])
"""


## Clustering

## OLD / OUTDATED CODE

In [ ]:
"""
Interaction dataset loaded for homo sapiens (HS) related data
Saving data with int. ids from id_prot
Raw Data has the headings: [protein1	protein2	neighborhood	fusion	cooccurence	coexpression	experimental	database	textmining	combined_score]

Runtime
Number of rows in HS protein links dataset: 13715404
Chunksize; reading one line per value 
Runtime when chunksize=200000; 
"""
chunksize = 200000
LinesRead = 0 
HS_int_df = pd.DataFrame() #df for storing
id_prot = joblib.load("./joblib_vars/id_prot.joblib")
logfile = open("./logs/IntRawData_logs.txt", "w")

for chunk in pd.read_csv("https://stringdb-downloads.org/download/protein.links.detailed.v12.0/9606.protein.links.detailed.v12.0.txt.gz", compression="gzip", chunksize=chunksize, sep=" "):
    print(f"Time: {time.ctime(time.time())}\tReading lines {LinesRead} - {LinesRead+chunksize}...", file=logfile)
    print(f"Time: {time.ctime(time.time())}\tReading lines {LinesRead} - {LinesRead+chunksize}...")
    chunk["protein1"] = chunk["protein1"].map(id_prot) #Changing names to id's using id_prot
    chunk["protein2"] = chunk["protein2"].map(id_prot) #Changing names to id's using id_prot
    HS_int_df = pd.concat([HS_int_df, chunk], axis=1)
    LinesRead += chunksize
    print(f"Time: {time.ctime(time.time())}\tLines read {LinesRead}/13,715,404", file=logfile)
    print(f"Time: {time.ctime(time.time())}\tLines read {LinesRead}/13,715,404")

joblib.dump(HS_int_df,"./mod_data/HS_int_df.joblib")
logfile.close()

### Setting the color of the progress bar

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

### Running the class

In [3]:
a = cl.interaction_network()
a.create_encoding_dict()
a.load_data()

Creating encoding table
Fetching data
Cropping data


Parsing data:   0%|          | 0/5847852 [00:00<?, ?it/s]